## Practice

1. Create a decorator that will check types. It should take a function with arguments and validate inputs with annotations.

Example:

    @check_types
    def add(a: int, b: int) -> int:
        return a + b

    add(1, 2)
    > 3

    add(1, "2")
    > TypeError: Argument b must be int, not str

In [483]:
'''
функція перевіряє на рівність таких типів, 
як list[int], tuple[tuple[str]], dict[str, int]
Щось на зразок list[int, str, float] не передбачається
'''
def check_generic_type(tp, obj):
    if tp.__origin__ == type(obj):
        dt = tp.__args__  # data types
        if type(obj) == dict:
            # змінні суто для читабельності
            keys = all(type(a) == dt[0] for a in obj.keys())
            vals = all(type(a) == dt[1] for a in obj.values())
            return keys and vals
        elif type(dt[0]) == type:
            return all(type(a) == dt[0] for a in obj)
        else:
            return all(check_generic_type(dt[0], a) for a in obj)
    else:
        return False


def check_inv_arg(arg, ann_type, name):
    msg = 'TypeError: Argument {arg} must be {ann_type}, not {act_type}'
    if type(ann_type) == type:
        if ann_type != type(arg):
            return msg.format(arg=name, ann_type=ann_type, act_type=type(arg))
    else:
        is_ok = check_generic_type(ann_type, arg)
        if not is_ok:
            return msg.format(arg=name, ann_type=ann_type, act_type=type(arg))


def check_types(func):
    def wrapper(*args, **kwargs):
        type_info = func.__annotations__
        # functions with *args, **kwargs
        try:
            func(*args, **kwargs, test=42)  # тест на *args, **kwargs у самі функції
            # тип для args і kwargs, в анотації вказується один на всіх
            ann_type = [i for i in type_info.values()]
            for arg in args:
                msg = check_inv_arg(arg, ann_type[0], arg)
                if msg:
                    print(msg)
                    return None
            for name, arg in kwargs.items():
                msg = check_inv_arg(arg, ann_type[1], name)
                if msg:
                    print(msg)
                    return None
        except:  # тут всі інші функції, без *args, **kwargs
            for arg, (name, ann_type) in zip(args, type_info.items()):
                msg = check_inv_arg(arg, ann_type, name)
                if msg:
                    print(msg)
                    return None
        return func(*args, **kwargs)
    return wrapper

In [493]:
@check_types
def add(a: int, b: int) -> int:
    return a + b


@check_types
def compute_difference(first: list[int], second: list[int]) -> tuple:
    return ([i for i in first if i not in second], [i for i in second if i not in first])


@check_types
def smth_different(*a: list[str], **kw: int):
    return 'No errors'

In [492]:
print('add -> ', add(1, '2'))
print('compute_difference -> ', compute_difference([2133, 1311, 42], ['23', 42]))
print('smth_different -> ', smth_different(['py', 'thon', 'add'], v=3, vv='11'))

TypeError: Argument b must be <class 'int'>, not <class 'str'>
add ->  None
TypeError: Argument second must be list[int], not <class 'list'>
compute_difference ->  None
TypeError: Argument vv must be <class 'int'>, not <class 'str'>
smth_different ->  None


## 2. Write a decorator that will calculate the execution time of a function.

Example:
    
        @calculate_execution_time
        def add(a: int, b: int) -> int:
            return a + b
    
        add(1, 2)
        > 3
        > Execution time: 0.0005 seconds

In [494]:
from datetime import datetime

In [523]:
def calculate_execution_time(func):
    def wrapper(*args, **kwargs):
        start = datetime.now()
        func(*args, **kwargs)
        finish = datetime.now()
        # microseconds, а не seconds, щоб були нецілі і більш точні значення
        return f'Exetution time of function "{func.__name__}": {(finish - start).microseconds/10**6} seconds'
    return wrapper

In [524]:
@calculate_execution_time
def t(n):
    for i in range(n):
        i

In [525]:
t(10**8)

'Exetution time of function "t": 0.43321 seconds'